<h1 align="center">Round 6 - Generative Adversarial Networks</h1>


This notebook is a part of teaching material for CS-EJ3311 - Deep Learning with Python 13.09.-17.12.2021\
Aalto University (Espoo, Finland)\
fitech.io (Finland)

In this notebook, we introduce our first generative model - Generative Adversarial Networks (GANs). Proposed in the year 2014 by Ian Goodfellow and colleagues, this model quickly gained popularity among deep learning practitioners and the general public due to very impressive results in image generation. Many apps that apply filters changing appearance, age, gender use GANs as their base method. GANs are used to generate realistic and cartoon-like images and videos, can create paintings and music, generate images from text description, and much more.\
First we discuss how generative models differs from models we learned before. We will go through some impressive GANs applications and the basic idea behind this architecture. Next, we will build simple GANs for data and image generation. 

### Learning goals

- understanding the general idea behind GAN and GAN applications
- understanding how to build a GAN model 
- understanding how to train a GAN model 

### Recommended reading

- Chapter 8.5 of "Deep Learning with Python" by F.Chollet. available via Aalto library [click here](https://primo.aalto.fi/discovery/search?query=any,contains,deep%20learning%20with%20python&tab=LibraryCatalog&search_scope=MyInstitution&sortby=date_d&vid=358AALTO_INST:VU1&facet=frbrgroupid,include,9062037433404512326&lang=en&offset=0)
- Chapter 17 of "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow" by Aurélien Géron. available via Aalto library [click here](https://primo.aalto.fi/discovery/search?query=any,contains,Hands-On%20Machine%20Learning%20with%20Scikit-Learn,%20Keras,%20and%20TensorFlow&tab=LibraryCatalog&search_scope=MyInstitution&vid=358AALTO_INST:VU1&lang=en&offset=0)
- Chapter 20.10 of "Deep Learning" by I.Goodfellow [click here](https://www.deeplearningbook.org)
- ["NIPS 2016 Tutorial: Generative Adversarial Networks"](https://arxiv.org/pdf/1701.00160.pdf)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt 
import IPython.display as ipd
from IPython.display import IFrame

## Generative Adversarial Nets

So far, our course focused on deep learning methods that predict the label of the datapoint. The label was typically associated with a certain class or category ("cats", "dogs"). In this round, we will learn about different type of deep learning methods. Instead of classifying data points (e.g., into "cat" or "dog" images), we tune the ANN to ***produce*** data that is very similar to the training or "target" set. Roughly speaking, we want the ANN to learn the statistical properties of the data points in a given training set. If the training set consists of dog images, then we want the ANN to learn how a typical dog image looks like. 

In a 2014 paper ["Generative Adversarial Nets"](https://arxiv.org/pdf/1406.2661.pdf) Ian Goodfellow et al. introduced a generative model inspired by the "Zero-Sum Game" principle from game theory. In the "Zero-Sum Game" with two palyers, one player's gain is equal to another player's loss, resulting in the net change being zero. In the context of deep learning, players are neural networks, which compete against each other. First ANN is a generator ANN, or simply generator, $G$, which reads in realizations $z^{(i)}$ of a latent (random) vector and generates a datapoint $G(z^{(i)})$. The second ANN, $D$, serves as a discriminator ANN. The discriminator reads in an image and predicts if this image has been obtained from the generator $G$ or from the "original" training dataset. The "fake" data is the data  $G({z}^{(i)})$ produced by the generator network and the "real" data is the training data  ${x}^{(i)}$ we provide to the discriminator $D$. 

<img src="https://i.imgur.com/0vuUsY0.png"  width=800 />


Above you see illustration ([image source](https://stefan-jansen.github.io/machine-learning-for-trading/21_gans_for_synthetic_time_series/)) of Generative Adversarial Network or GAN. First, a random vector is formed by sampling from a normal distribution. This random vector of fixed size is then fed to the generator network. The generator outputs a sample (fake sample or fake data point). Together with real data, fake samples are fed to a discriminator network, which classifies them into two categories: "fake" and "real".\
Initially, the samples produced by the generator are basically noise and the discriminator easily classifies them as "fake". After a few iterations of the SGD algorithm generator's weights are tuned in a such way that it will produce samples that are more similar to the real data.\
When the generator produces samples closely resembling target data, the generator's loss (error) is getting smaller, but the discriminator's loss is getting larger. Thus, the generator is competing against its opponent or adversary (hence, Generative Adversarial Network) and the final goal is to produce such samples, that discriminator will not be able to distinguish from real data and the classification accuracy of the discriminator will be around 0.5 (50% chance). 
More formally, the algorithm for GANs is described in the paper ["Generative Adversarial Nets"](https://arxiv.org/pdf/1406.2661.pdf):

<img src="../../../coursedata/R6/gan_algo.png" width=700/>


We can see, that there are two distinct parts or phases in the presented algorithm:

- Phase 1 Discriminator training

1. Fetch batch of real data. 
2. Sample random vector, feed to the generator to get fake data.
3. Feed real and fake data to the discriminator for classification.
4. Compute loss and backpropagate the error/loss through **discriminator**. Adjust discriminator's weights.

- Phase 2 Generator training

1. Sample random vector, feed to the generator to get fake data.
3. Feed fake data to the discriminator for classification.
4. Compute loss and backpropagate the error/loss through **generator**. Adjust generator's weights.

The discriminator is penalized for classifying fake data as "real". In contrast, the generator is penalized if the discriminator classified fake data as "fake".\
To get more comfortable with the GAN model you can try interactive GAN training at https://poloclub.github.io/ganlab/.

## GAN applications


### Realistic image generation

One popular application is the generation of realistic images and specifically human faces. Although we do not recognize it consciously, we, humans, are very sensitive to facial features, proportions, and expressions. It is quite a difficult task for ANN to produce synthetic images and videos of human faces that would fool humans. Nevertheless, deep learning generative models improved vastly since the first GANs were built. Below you see an illustration from ["The AI Index 2021 Annual Report"](https://aiindex.stanford.edu/wp-content/uploads/2021/11/2021-AI-Index-Report_Master.pdf) - set of images generated by GANs in different years. 
 

<img src="https://miro.medium.com/max/1400/1*zlQpCy8gdNTNt0A6q6d0pA.jpeg">

You can test yourself and try to find which image is real and which is not on the website https://www.whichfaceisreal.com/results.php?r=1&p=0&i1=46594.jpeg&i2=image-2019-02-18_094509.jpeg


Now it is possible not only to generate synthetic images, but also videos. For example, deep fake methods are used to generate fake videos with celebrities for entertainment. As an example here is a deep fake of [Queen of the United Kingdom](https://www.youtube.com/watch?v=IvY-Abd2FfM) and actor [Morgan Freeman](https://www.youtube.com/watch?v=oxXpB9pSETo). 


### Image-to-Image Translation

Another popular application is image transformations or image-to-image translation. Such transformations may be adding a style of a famous painter to a photo, application of zebra's stripe pattern to a horse, changing a summer photo to a winter photo. Below are such examples generated by [CycleGAN](https://junyanz.github.io/CycleGAN/).

<img src="https://camo.githubusercontent.com/16fa02525bf502bec1aac77a3eb5b96928b0f25d73f7d9dedcc041ba28c38751/68747470733a2f2f6a756e79616e7a2e6769746875622e696f2f4379636c6547414e2f696d616765732f7465617365725f686967685f7265732e6a7067" />


Similar to style transfer real images can be transformed to cartoon-style corresponding images. There are some examples from paper ["CartoonGAN: Generative Adversarial Networks for Photo Cartoonization"](https://openaccess.thecvf.com/content_cvpr_2018/papers/Chen_CartoonGAN_Generative_Adversarial_CVPR_2018_paper.pdf):

<img src="../../../coursedata/R6/cartoongan.png" />


It is also possible to perform style transfer on videos. Below are a few examples:

In [ ]:
# https://github.com/lengstrom/fast-style-transfer
IFrame(src="https://www.youtube.com/embed/xVJwwWQlQ1o", width=560, height=315)

In [ ]:
# http://arxiv.org/abs/1604.08610
IFrame(src="https://www.youtube.com/embed/Khuj4ASldmU?start=45", width=560, height=315)

### Text-to-Image Translation


There are GAN models that are able to generate images only from the text description. For example [StackGAN](https://github.com/hanzhanggit/StackGAN) or [AttnGAN](https://github.com/taoxugit/AttnGAN). Below you can see examples from StackGAN.

<img src="https://raw.githubusercontent.com/hanzhanggit/StackGAN/master/examples/bird3.jpg"> 

### Art

Generative models found application not only in hard science but also in art and literature.
One example of such collaboration is visual artist and software engineer Helena Sarin. From [nvidia article](https://www.nvidia.com/en-us/research/ai-art-gallery/artists/helena-sarin/):

<div class="blockquote-container">
    <blockquote class="ludwig">
     "... Helena collected and photographed more than 1000 foliage leaves to use as part of her dataset. She modified CycleGAN and her own modified version of PGAN, finding that SOTA performs poorly on small, diverse datasets."
    </blockquote>
</div>

<img src="https://www.nvidia.com/content/dam/en-zz/Solutions/deep-learning/ai-art-gallery/gtc21-ai-art-gallery-helena-sarin-process-3-2c50-d.jpg">



### Synthetic music

GAN models are capable of generating data of different types, including audio signals. For example, GANSynth learns to produce individual instrument notes, and MuseGAN is trained to generate polyphonic music of multiple tracks (instruments). It can generate music either from scratch or by accompanying a track given by the user.

You can find more information on these types of GAN models and produced audio here:

- [GANSynth](https://storage.googleapis.com/magentadata/papers/gansynth/index.html)
- [MuseGAN](https://salu133445.github.io/musegan/) 

Below is an audio file with generated samples of polyphonic music by MuseGAN:

In [ ]:
ipd.Audio('../../../coursedata/R6/best_samples.mp3') # load an audio file

More resources with GAN applications:

- [art-DCGAN, github repo](https://github.com/robbiebarrat/art-DCGAN#landscape-gan)
- list ["41 Creative Tools to Generate AI Art"](https://aiartists.org/ai-generated-art-tools)
- list of papers & repos [The GAN Zoo](https://github.com/hindupuravinash/the-gan-zoo#the-gan-zoo) 
- [Curated list of GAN applications and demons](https://github.com/nashory/gans-awesome-applications)
- book ["Generative Deep Learning: Teaching Machines to Paint, Write, Compose, and Play"](https://www.amazon.com/Generative-Deep-Learning-Teaching-Machines/dp/1492041947?dchild=1&keywords=%22reinforcement+learning%22&qid=1616529606&s=books&sr=1-4&linkCode=sl1&tag=petekistler-20&linkId=ad36a33f10ab1747ae2a2158c4714336&language=en_US&ref_=as_li_ss_tl)

<div class=" alert alert-warning">
    <h3><b>Student task. </b>Generate samples from quadratic distribution.</h3>
    
In this task, we implement a simple GAN model for approximating samples from the quadratic distribution. The target (training) data consist of pairs of values $(x,y)$, where $x$ is a real-valued number and $ y = 10 + {x}^{2}$.\
We can sample from target data with custom function `sample_true_distr()`.

Your tasks are:
    
- [Create tf.data.Dataset](#ds)
- [Build a generator](#G)
- [Build a discriminator](#D)  
    
Code adapted from https://blog.paperspace.com/implementing-gans-in-tensorflow/
</div>

In [ ]:
def sample_true_distr(n=10000):
    """
    Function to generate samples from the quadratic distribution.
    Takes as input sample size `n` and returns set of n-pairs (x,y),
    where x is a real number and y is a square of x plus a constant.
    """
    
    # set random seed for reproducibility
    np.random.seed(42)
    # draw samples from normal distribution
    x = 10*(np.random.random_sample((n,))-0.5)
    # compute y
    y = 10 + x*x

    return np.array([x, y]).T

# create a dataset with samples from the quadratic distribution
target_data = sample_true_distr()
# plot the first 32 datapoints
plt.scatter(target_data[:32,0],target_data[:32,1])
plt.xlabel("x", fontsize=20)
plt.ylabel("y", fontsize=20)
plt.show()

<a id='ds'></a>
<div class=" alert alert-warning">
    
Create tf.data.Dataset object:
    
- create tf.data.Dataset by passing array `target_data` to `from_tensor_slices()` method. See examples [here](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices)
- map dataset with custom function `cast_func()`
- shuffle samples. Set `buffer_size` to 1000 and `seed` arg to 42.
- add batching. Batch size should be equal to varibale `batch_size`, set `drop_remainder` to True
- prefetch batch. Set `buffer_size` to 1
    
    
You can find the list of methods for tf.data.Datasets at https://www.tensorflow.org/api_docs/python/tf/data/Dataset.
</div>

In [ ]:
batch_size = 32

def cast_func(ds):
    
    """
    Function that takes tensor as input and
    returns it as a tensor of dtype tf.float32.
    """
    ds = tf.cast(ds, tf.float32)
    return ds

# YOUR CODE HERE
raise NotImplementedError()

# dataset = ...   # create tf.data.Dataset
# dataset = ...   # map
# dataset = ...   # shuffle
# dataset = ...   # batching
# dataset = ...   # prefetch

In [ ]:
# sanity checks 
ds = list(dataset.take(1))[0][0]
np.testing.assert_almost_equal(ds,[-4.6311307, 31.44737], decimal=4)

print("Sanity checks passed!")

In [ ]:
# this cell is for tests


<a id='G'></a>
<div class=" alert alert-warning">
    
Build a generator ANN. The generator should take input of shape [codings_size]. Use layers:
 
- Dense with 16 units and leaky relu activation `tf.nn.leaky_relu`
- Dense with 16 units and leaky relu activation 
- Dense with 2 output units (values for x and y) 

Add name to the model as `generator = tf.keras.models.Sequential([...], name="Generator")`
</div>

In [ ]:
# size of the random vector
codings_size = 10

# YOUR CODE HERE
raise NotImplementedError()

# generator = ...
generator.summary()

In [ ]:
# Perform some sanity checks on the solution
assert len(generator.layers) == 3, "There should be 3 layers!"
assert generator.name == 'Generator', "Name generator as 'Generator'"

print("Sanity checks passed!")

In [ ]:
# this cell is for tests


<a id='D'></a>
<div class=" alert alert-warning" id="D">
    
Build a discriminator. Discriminator should take input of shape [2] and return the probability of sample (pair {x,y}) being fake or real. Use layers:
 
- Dense with 16 units and leaky relu activation 
- Dense with 16 units and leaky relu activation 
- Dense with 1 output unit and sigmoid activation.
    
Add name to the model as `discriminator = tf.keras.models.Sequential([...], name="Discriminator")`
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# discriminator = ...
discriminator.summary()

In [ ]:
# Perform some sanity checks on the solution
assert len(discriminator.layers) == 3, "There should be 3 layers!"
assert discriminator.name == 'Discriminator', "Name discriminator as 'Discriminator'"

print("Sanity checks passed!")

In [ ]:
# this cell is for tests


In [ ]:
# combine both ANNs into one model `gan`
gan = tf.keras.models.Sequential([generator, discriminator])

tf.keras.utils.plot_model(
    gan,
    show_shapes=True, 
    show_layer_names=True
)

You may notice, that we used leaky relu instead of relu in both, generator and discriminator. In general, training GAN as it was described originally is very difficult. Many hyperparameters had to be tuned just by trial-and-error. It was found experimentally, that relu activation, especially in discriminator ANN, destabilizes training and leads to poor results. \
You can find more hints on training GANs in the article ["Tips for Training Stable Generative Adversarial Networks"](https://machinelearningmastery.com/how-to-train-stable-generative-adversarial-networks/).

<a id='compile'>Compiling discriminator and GAN.</a>

Now we can compile the models. First, we compile the discriminator. By default, all parameters of the Keras model are trainable, which means that discriminator weights are tuned during training.\
In contrast, we set `discriminator.trainable = False` to fix discriminator weights during `gan` model training. We need to fix discriminator weights in order to train the generator and to only update the generator's weights. \
In other words, when we train discriminator as a separate model, its' parameters are adjusted during training. On the other hand, when the discriminator is trained as a part of the `gan` model, its' weights are fixed.

**Note!** Set `model.trainable` False or True **BEFORE** compiling the model. For example, if we set `model.trainable = False` after compilation, it will not have an effect. Even though printing `model.trainable ` will return `False`, the model in fact will be trainable.

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
# set training flag 
training=True

In [ ]:
# this cell setting flag training=False


Now we are almost ready to train our first GAN model. We cannot use `.fit()` method in this case, instead we write  custom training function `train_gan()`. This function takes as input model (GAN), batch size, codings size, and n.o. epochs. Codings size is the size of a random (noise) vector that the generator takes as input. 

We will save generated samples for later plotting in an array `saved_samples`.\
There are two training phases involved: phase 1 for training discriminator and phase 2 for training generator. 

Phase 1 consist of the following steps:

1. sample random vectors of shape (batch_size, codings_size) from a normal distribution
2. feed the random vectors to the generator, which will output generated samples
3. concatenate fake and real samples in one set
4. create labels for fake ($y=0$) and real ($y=1$) samples
5. use the `train_on_batch(features, labels)` method to train the discriminator. This method runs a single gradient update on a single batch of data. Reference to [Keras documentaion](https://keras.io/api/models/model_training_apis/#trainonbatch-method).

Phase 2 consists of the following steps:

1. sample random vectors of shape (batch_size, codings_size) from a normal distribution
2. create labels for fake samples $y=1$
3. use the `train_on_batch(features, labels)` method to train gan. By applying this method to the GAN model we will tune the weights of the generator. Parameters of discriminator are not affected, as previously we set `discriminator.trainable = False` before compiling gan.

For training discriminator and generator we use binary cross-entropy loss, which for one sample is: 

\begin{equation*}
\mathcal L = y \cdot ln(\hat y) + (1-y)\cdot ln(1-\hat y)
\end{equation*}

By switching labels from $y=0$ to $y=1$ for fake images during the generator training phase, we achieve the same result as in the original algorithm described in the paper "Generative Adversarial Nets".
For discriminator, the loss will be minimized when predicted labels for real images are $\hat y=1$ and predicted labels for fake images are $\hat y=0$. For generator, the loss will be minimal when predicted labels for fake images are $\hat y=1$, due to the fact that we assigned labels $y=1$ to the fake images for generator training.

In [ ]:
%%time 

def train_gan(gan, dataset, batch_size, codings_size, n_epochs=90):
    saved_samples = np.zeros((int(n_epochs/10),2,batch_size,2))
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):             
        for X_batch in dataset:
            
            # phase 1 - training the Discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            gen_samples = generator(noise)
            X_fake_and_real = tf.concat([gen_samples, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.train_on_batch(X_fake_and_real, y1)
            
            # phase 2 - training the Generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            gan.train_on_batch(noise, y2)
            
        if epoch%10 == 0:
            print("Epoch {}/{}".format(epoch, n_epochs)) 
            saved_samples[int(epoch/10),0,:,:] = X_batch
            saved_samples[int(epoch/10),1,:,:] = gen_samples

    return saved_samples

if training:                        
    saved_samples = train_gan(gan, dataset, batch_size, codings_size)

In [ ]:
if training:
    plt.figure(figsize=(8, 8))

    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        # plot real samples
        plt.scatter(saved_samples[i,0,:,0],saved_samples[i,0,:,1])
        # plot generated (fake) samples
        plt.scatter(saved_samples[i,1,:,0],saved_samples[i,1,:,1])
        plt.axis("off")   
    plt.show()

### "Hello world!" of DCGAN: MNIST dataset

We can generate images using the same Generator-Discriminator ANNs, but using convolutional layers instead of Dense layers. This type of GAN is called deep convolutional generative adversarial network (DCGAN). Find more information in the original paper ["Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks."](https://arxiv.org/pdf/1511.06434.pdf).

<img src="../../../coursedata/R6/gan.gif" width=600 />

### Transposed convolution.

To build a GAN for image generation we use the same principles as in the first example, but first, we need to decide how to create a 2D image from a 1D tensor. We can imagine that a given input is a result of some convolution and we need to "reverse" this convolution and recover the shape of the initial feature map. To achieve this we use so-called **transposed convolution**, a process very similar to the convolution you are familiar with. You can find more information on transposed convolution in the section "4.2 Transposed convolution" of an article ["A guide to convolution arithmetic for deep learning"](https://arxiv.org/abs/1603.07285).\
To this end it's enough to know that transposed convolution is implemented with [`tf.keras.layers.Conv2DTranspose`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose). Unfortunately, TensorFlow documentation is somewhat confusing on how to compute output shape, but the general rule is (see discussion [here](https://datascience.stackexchange.com/questions/26451/how-to-calculate-the-output-shape-of-conv2d-transpose)):

With `padding="same"`:\
out_dimension = in_dimension * stride 

With `padding="valid"`:\
out_dimension = (in_dimension-1) * stride + kernel_size 


For example, for input of shape (batch_size, 7, 7, 128) and transposed convolution `tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same")`

out_dimension = in_dimension * stride = 7*2 = 14

and for `tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="valid")`

out_dimension = (in_dimension-1) * stride + kernel_size = (7-1)*2+5 = 17

Let's confirm our calculations by using TensorFlow:

In [ ]:
cvt_same = tf.keras.models.Sequential([
    tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same",input_shape=(7, 7, 128)),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="same",
                                 activation="tanh")
])

tf.keras.utils.plot_model(
    cvt_same,
    show_shapes=True, 
    show_layer_names=True
)

In [ ]:
cvt_valid = tf.keras.models.Sequential([
    tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="valid",input_shape=(7, 7, 128))
])

tf.keras.utils.plot_model(
    cvt_valid,
    show_shapes=True, 
    show_layer_names=True
)

We use the MNIST dataset of handwritten digits, but instead of scaling pixel values to the (0,1) range, we scale them to the (-1,1) range. Similarly, we scale generator output to (-1,1) range by using tanh activation function in the last layer.

In [ ]:
if training:
    (X_train, y_train), (_, _) = tf.keras.datasets.mnist.load_data()  # load training set
    X_train = X_train.reshape(-1, 28, 28, 1)/255 * 2. - 1.  # reshape and rescale
    X_train = tf.cast(X_train, tf.float32)  # change data type

In [ ]:
if training:
    batch_size = 32
    dataset = tf.data.Dataset.from_tensor_slices(X_train)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

<div class=" alert alert-warning">
    <h3><b>Student task. </b>Generator.</h3>
    
Your task is to build a generator. The generator is a sequential model with:
    
- Dense layer with 6272 (=7\*7\*128) units and `input_shape=[codings_size]`. 
- Reshape layer `tf.keras.layers.Reshape([7, 7, 128])`
- Batch Normalization layer
- Conv2DTranspose layer with 64 kernels, kernel size 5, strides 2, padding "same" and selu activation
- Batch Normalization layer
- Conv2DTranspose layer with 1 output kernel, kernel size 5, strides 2, padding "same" and tanh activation
    
Add name to the model as `cv_generator = tf.keras.models.Sequential([...], name="Generator")`
</div>

In [ ]:
codings_size = 100

# YOUR CODE HERE
raise NotImplementedError()

# cv_generator = ...

In [ ]:
# Perform some sanity checks on the solution
assert len(cv_generator.layers) == 6, "There should be 6 layers!"
assert cv_generator.name == 'Generator', "Name the generator as 'Generator'"

print("Sanity checks passed!")

In [ ]:
# this cell is for tests


<div class=" alert alert-warning">
    <h3><b>Student task. </b>Discriminator.</h3>
    
Your task is to build a discriminator. Discriminator should take as input image generated by generator and output probability of the image being fake or real.
    
Use:
    
- Conv2DTranspose layer with 64 kernels, kernel size 5, strides 2, padding "SAME" and leaky relu activation (alpha=0.2) 
- Conv2DTranspose layer with 128 kernels, kernel size 5, strides 2, padding "SAME" and activation leaky relu activation (alpha=0.2)  
- Flatten layer    
- Dense output layer      
    
Add name to the model as `cv_discriminator = tf.keras.models.Sequential([...], name="Discriminator")`
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# cv_discriminator = ...

In [ ]:
# Perform some sanity checks on the solution
assert len(cv_discriminator.layers) == 4, "There should be 4 layers!"
assert cv_discriminator.name == 'Discriminator', "Name discriminator as 'Discriminator'"

print("Sanity checks passed!")

In [ ]:
# this cell is for tests


In [ ]:
# build a GAN model
cv_gan = tf.keras.models.Sequential([cv_generator, cv_discriminator])

tf.keras.utils.plot_model(
    cv_gan,
    show_shapes=True, 
    show_layer_names=True
)

<div class=" alert alert-warning">
    <h3><b>Student task. </b>Compile discriminator and gan.</h3>
    
Compile discriminator `cv_discriminator` and GAN `cv_gan` model as was explained in ["Compile discriminator and GAN"](#compile).
</div>

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

# cv_discriminator...
# cv_discriminator...
# cv_gan...

In [ ]:
# Perform some sanity checks on the solution
assert cv_discriminator.trainable==False, "The discriminator should not be trainable!"
assert cv_gan.trainable == True, "The GAN should be trainable!"
assert cv_discriminator.loss == 'binary_crossentropy', "The discriminator loss should be binary CE"
assert cv_gan.loss == 'binary_crossentropy', "The GAN loss should be binary CE"

print("Sanity checks passed!")

In [ ]:
# this cell is for tests


In [ ]:
# function for plotting images outputted by generator
# code source https://github.com/ageron/handson-ml2/blob/master/17_autoencoders_and_gans.ipynb

def plot_multiple_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")    

In [ ]:
%%time 

test_random_vector = tf.random.normal(shape=[batch_size, codings_size])

def train_gan(gan, dataset, batch_size, codings_size, n_epochs=2):
    
    generator, discriminator = gan.layers
    itr=0
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs)) 
        for X_batch in dataset:
            # Phase 1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            
            # Phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
            
            if (itr%100 == 0):  
                gen_images = generator.predict(test_random_vector)
                plot_multiple_images(gen_images, 8)
                plt.show()
                         
            itr+=1

if training:                   
    train_gan(cv_gan, dataset, batch_size, codings_size)

<div class=" alert alert-warning">
    <h3>Question 1.</h3>
    
Choose the correct statement. GANs are typically used for:

1. Predicting real-valued labels of data points
2. Classifying images
3. Generating data similar to a given "reference" or "target" dataset
4. Clustering data
    
</div>

In [ ]:
# remove the line raise NotImplementedError() before testing your solution and submitting code

# YOUR CODE HERE
raise NotImplementedError()

# answer_1 = ...

In [ ]:
# This cell is for tests
assert answer_1 in [1, 2, 3, 4], '"answer" Value should be an integer between 1 and 4.'
print('Sanity check tests passed!')


<div class=" alert alert-warning">
    <h3>Question 2.</h3>
    
Choose the correct statement. Generative Adversarial Networks:

1. is a feed-forward densely connected network
2. is a convolutional neural network
3. is a layer of an ANN
4. consists of two ANNs, one of them serving as a data generator and the other one serving as discriminator between "fake" and "real" datapoints
    
</div>

In [ ]:
# remove the line raise NotImplementedError() before testing your solution and submitting code

# YOUR CODE HERE
raise NotImplementedError()

# answer_2 = ...

In [ ]:
# This cell is for tests
assert answer_2 in [1, 2, 3, 4], '"answer" Value should be an integer between 1 and 4.'
print('Sanity check tests passed!')


<div class=" alert alert-warning">
    <h3>Question 3.</h3>
    
Choose the correct statement. 
    
During the **Discriminator** training phase, the discriminator is fed with datapoints:

1. that are obtained from the generator ANN ("fake") and from the "target" dataset ("real"). Label for "fake" datapoints is $y=1$ and for "real" datapoints $y=0$.
2. that is obtained only from the generator ANN ("fake"). Label for "fake" datapoints is $y=1$.
3. that are obtained from the generator ANN ("fake") and from the "target" dataset ("real"). Label for "fake" datapoints is $y=0$ and for "real" datapoints $y=1$.
4. that is obtained only from the generator ANN ("fake"). Label for "fake" datapoints is $y=0$.
        
</div>

In [ ]:
# remove the line raise NotImplementedError() before testing your solution and submitting code

# YOUR CODE HERE
raise NotImplementedError()

# answer_3 = ...

In [ ]:
# This cell is for tests
assert answer_3 in [1, 2, 3, 4], '"answer" Value should be an integer between 1 and 4.'
print('Sanity check tests passed!')


<div class=" alert alert-warning">
    <h3>Question 4.</h3>
    
During the **Generator** training phase, the discriminator is fed with datapoints:

1. that are obtained from the generator ANN ("fake") and from the "target" dataset ("real"). Label for "fake" datapoints is $y=1$ and for "real" datapoints $y=0$.
2. that is obtained only from the generator ANN ("fake"). Label for "fake" datapoints is $y=1$.
3. that are obtained from the generator ANN ("fake") and from the "target" dataset ("real"). Label for "fake" datapoints is $y=0$ and for "real" datapoints $y=1$.
4. that is obtained only from the generator ANN ("fake"). Label for "fake" datapoints is $y=0$.

    
</div>

In [ ]:
# remove the line raise NotImplementedError() before testing your solution and submitting code

# YOUR CODE HERE
raise NotImplementedError()

# answer_4 = ...

In [ ]:
# This cell is for tests
assert answer_4 in [1, 2, 3, 4], '"answer" Value should be an integer between 1 and 4.'
print('Sanity check tests passed!')


<div class=" alert alert-warning">
    <h3>Question 5.</h3>
    
Choose the correct statement. During **generator** training phase:

1. generator weights are fixed (not trainable)
2. discriminator weights are fixed
3. both, generator and discriminator weights are fixed
4. both, generator and discriminator weights are trainable
    
</div>

In [ ]:
# remove the line raise NotImplementedError() before testing your solution and submitting code

# YOUR CODE HERE
raise NotImplementedError()

# answer_5 = ...

In [ ]:
# This cell is for tests
assert answer_5 in [1, 2, 3, 4], '"answer" Value should be an integer between 1 and 4.'
print('Sanity check tests passed!')


In [ ]:
# This cell is for tests


In [ ]:
# This cell is for tests
